In [1]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 6.9 MB/s eta 0:00:00


In [2]:
import langgraph.graph as lg

print(dir(lg))

['END', 'MessageGraph', 'MessagesState', 'START', 'StateGraph', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_branch', '_node', 'add_messages', 'message', 'state']


In [ ]:
[name for name in dir(lg) if not name.startswith("_")]

['END',
 'MessageGraph',
 'MessagesState',
 'START',
 'StateGraph',
 'add_messages',
 'message',
 'state']

In [ ]:
from langgraph.graph import StateGraph, MessageGraph, MessagesState

print(dir(StateGraph))
print(dir(MessageGraph))
print(dir(MessagesState))


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_schema', '_all_edges', 'add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__deprecated__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr_

In [ ]:
import inspect

def list_methods(cls):
    return [name for name, obj in inspect.getmembers(cls, inspect.isfunction) if not name.startswith("_")]

print("StateGraph:", list_methods(StateGraph))
print("MessageGraph:", list_methods(MessageGraph))
print("MessagesState:", list_methods(MessagesState))


StateGraph: ['add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
MessageGraph: ['add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
MessagesState: []


In [ ]:
import importlib
import inspect
import dataclasses
import typing as t

# --- Optional schema detection support ---
try:
    from pydantic import BaseModel as _PydanticBaseModel
except Exception:
    _PydanticBaseModel = None

try:
    import typing
    _TypedDictMeta = typing.TypedDict.__class__
except Exception:
    _TypedDictMeta = None

# ------------------ helpers ------------------

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _is_function(obj: t.Any) -> bool:
    return inspect.isfunction(obj) or inspect.isbuiltin(obj)

def _is_class(obj: t.Any) -> bool:
    return inspect.isclass(obj)

def _is_module(obj: t.Any) -> bool:
    return inspect.ismodule(obj)

def _looks_like_typed_dict(cls: t.Any) -> bool:
    return _TypedDictMeta and isinstance(cls, _TypedDictMeta)

def _looks_like_pydantic_model(cls: t.Any) -> bool:
    return _PydanticBaseModel and inspect.isclass(cls) and issubclass(cls, _PydanticBaseModel)

def _looks_like_dataclass(cls: t.Any) -> bool:
    return dataclasses.is_dataclass(cls)

def _class_public_methods(cls: type) -> list[str]:
    methods = []
    for name, member in inspect.getmembers(cls):
        if not _is_public(name):
            continue
        if inspect.isfunction(member) or inspect.ismethoddescriptor(member):
            methods.append(name)
        elif isinstance(member, (classmethod, staticmethod)):
            methods.append(name)
    return sorted(set(methods))

def _class_public_attributes(cls: type) -> list[str]:
    attrs = []
    for name, member in inspect.getmembers(cls):
        if not _is_public(name):
            continue
        if _is_module(member) or _is_function(member):
            continue
        if inspect.ismethod(member) or inspect.ismethoddescriptor(member):
            continue
        attrs.append(name)
    return sorted(set(attrs))

def _short_type(obj: t.Any) -> str:
    return type(obj).__name__

def _safe_repr(obj: t.Any, maxlen: int = 80) -> str:
    try:
        r = repr(obj)
    except Exception:
        r = f"<{_short_type(obj)}>"
    if len(r) > maxlen:
        r = r[: maxlen - 1] + "…"
    return r

# ------------------ main API ------------------

def explore_module(module_name: str) -> dict:
    mod = importlib.import_module(module_name)
    classes, functions, constants = {}, [], []

    for name in dir(mod):
        if not _is_public(name):
            continue
        obj = getattr(mod, name)
        if _is_module(obj):
            continue
        if _is_class(obj):
            if _looks_like_pydantic_model(obj):
                schema_kind = "pydantic"
            elif _looks_like_dataclass(obj):
                schema_kind = "dataclass"
            elif _looks_like_typed_dict(obj):
                schema_kind = "typeddict"
            else:
                schema_kind = None
            classes[name] = {
                "schema_kind": schema_kind,
                "methods": _class_public_methods(obj),
                "attributes": _class_public_attributes(obj),
            }
        elif _is_function(obj):
            functions.append(name)
        else:
            constants.append(
                {"name": name, "type": _short_type(obj), "repr": _safe_repr(obj)}
            )

    return {
        "module": module_name,
        "classes": dict(sorted(classes.items())),
        "functions": sorted(functions),
        "constants": sorted(constants, key=lambda d: d["name"]),
    }

def pretty_print_report(report: dict) -> None:
    mod = report["module"]
    print(f"\n=== Module: {mod} ===")

    print("\nClasses:")
    if not report["classes"]:
        print("  (none)")
    else:
        for cls_name, meta in report["classes"].items():
            schema = f" [schema={meta['schema_kind']}]" if meta["schema_kind"] else ""
            print(f"  - {cls_name}{schema}")
            print(f"      methods: {meta['methods']}")
            print(f"      attributes: {meta['attributes']}")

    print("\nFunctions:")
    if not report["functions"]:
        print("  (none)")
    else:
        for fn in report["functions"]:
            print(f"  - {fn}")

    print("\nConstants / Objects / Sentinels:")
    if not report["constants"]:
        print("  (none)")
    else:
        for c in report["constants"]:
            print(f"  - {c['name']} : <{c['type']}> = {c['repr']}")

# ------------------ Example usage in notebook ------------------

# Example: explore langgraph.graph
report = explore_module("langgraph.graph")
pretty_print_report(report)


=== Module: langgraph.graph ===

Classes:
  - MessageGraph
      methods: ['add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
      attributes: []
  - MessagesState
      methods: ['clear', 'copy', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']
      attributes: []
  - StateGraph
      methods: ['add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
      attributes: []

Functions:
  - add_messages

Constants / Objects / Sentinels:
  - END : <str> = '__end__'
  - START : <str> = '__start__'


In [ ]:
# === Notebook-friendly module & package explorer with colored pretty print ===
from __future__ import annotations

import importlib
import inspect
import dataclasses
import typing as t
import pkgutil

# ---------- Optional schema detection support ----------
try:
    from pydantic import BaseModel as _PydanticBaseModel
except Exception:
    _PydanticBaseModel = None

try:
    import typing
    _TypedDictMeta = typing.TypedDict.__class__
except Exception:
    _TypedDictMeta = None


# ---------- ANSI color helpers (safe if colors unsupported) ----------
class _C:
    BOLD = "\033[1m"
    DIM = "\033[2m"
    RESET = "\033[0m"
    BLUE = "\033[34m"
    CYAN = "\033[36m"
    GREEN = "\033[32m"
    MAGENTA = "\033[35m"
    YELLOW = "\033[33m"
    GRAY = "\033[90m"

def _c(s: str, color: str) -> str:
    return f"{color}{s}{_C.RESET}"


# ---------- Introspection helpers ----------
def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _is_function(obj: t.Any) -> bool:
    return inspect.isfunction(obj) or inspect.isbuiltin(obj)

def _is_class(obj: t.Any) -> bool:
    return inspect.isclass(obj)

def _is_module(obj: t.Any) -> bool:
    return inspect.ismodule(obj)

def _looks_like_typed_dict(cls: t.Any) -> bool:
    return _TypedDictMeta and isinstance(cls, _TypedDictMeta)

def _looks_like_pydantic_model(cls: t.Any) -> bool:
    return _PydanticBaseModel and inspect.isclass(cls) and issubclass(cls, _PydanticBaseModel)

def _looks_like_dataclass(cls: t.Any) -> bool:
    return dataclasses.is_dataclass(cls)

def _class_public_methods(cls: type) -> list[str]:
    methods = []
    for name, member in inspect.getmembers(cls):
        if not _is_public(name):
            continue
        if inspect.isfunction(member) or inspect.ismethoddescriptor(member):
            methods.append(name)
        elif isinstance(member, (classmethod, staticmethod)):
            methods.append(name)
    return sorted(set(methods))

def _class_public_attributes(cls: type) -> list[str]:
    attrs = []
    for name, member in inspect.getmembers(cls):
        if not _is_public(name):
            continue
        if _is_module(member) or _is_function(member):
            continue
        if inspect.ismethod(member) or inspect.ismethoddescriptor(member):
            continue
        attrs.append(name)
    return sorted(set(attrs))

def _short_type(obj: t.Any) -> str:
    try:
        return type(obj).__name__
    except Exception:
        return "<unknown>"

def _safe_repr(obj: t.Any, maxlen: int = 80) -> str:
    try:
        r = repr(obj)
    except Exception:
        r = f"<{_short_type(obj)}>"
    if len(r) > maxlen:
        r = r[: maxlen - 1] + "…"
    return r


# ---------- Module explorer ----------
def explore_module(module_name: str) -> dict:
    """
    Import a module and return a structured dictionary:
      - classes (with schema_kind, methods, attributes)
      - functions
      - constants/objects/sentinels
    """
    mod = importlib.import_module(module_name)
    classes: dict[str, dict] = {}
    functions: list[str] = []
    constants: list[dict[str, str]] = []

    for name in dir(mod):
        if not _is_public(name):
            continue
        obj = getattr(mod, name)
        if _is_module(obj):
            continue

        if _is_class(obj):
            if _looks_like_pydantic_model(obj):
                schema_kind = "pydantic"
            elif _looks_like_dataclass(obj):
                schema_kind = "dataclass"
            elif _looks_like_typed_dict(obj):
                schema_kind = "typeddict"
            else:
                schema_kind = None
            classes[name] = {
                "schema_kind": schema_kind,
                "methods": _class_public_methods(obj),
                "attributes": _class_public_attributes(obj),
            }
        elif _is_function(obj):
            functions.append(name)
        else:
            constants.append(
                {"name": name, "type": _short_type(obj), "repr": _safe_repr(obj)}
            )

    return {
        "module": module_name,
        "classes": dict(sorted(classes.items(), key=lambda kv: kv[0].lower())),
        "functions": sorted(functions, key=str.lower),
        "constants": sorted(constants, key=lambda d: d["name"].lower()),
    }


# ---------- Pretty printer ----------
def pretty_print_report(report: dict) -> None:
    mod = report["module"]
    print(_c(f"\n=== Module: {mod} ===", _C.BOLD + _C.BLUE))

    print(_c("\nClasses:", _C.BOLD + _C.MAGENTA))
    if not report["classes"]:
        print("  (none)")
    else:
        for cls_name, meta in report["classes"].items():
            schema = f" [schema={meta['schema_kind']}]" if meta["schema_kind"] else ""
            print(_c(f"  - {cls_name}{schema}", _C.GREEN))
            methods = meta["methods"]
            attrs = meta["attributes"]
            print(_c("      methods:", _C.DIM), methods if methods else "[]")
            print(_c("      attributes:", _C.DIM), attrs if attrs else "[]")

    print(_c("\nFunctions:", _C.BOLD + _C.MAGENTA))
    if not report["functions"]:
        print("  (none)")
    else:
        for fn in report["functions"]:
            print(_c("  - ", _C.GREEN) + fn)

    print(_c("\nConstants / Objects / Sentinels:", _C.BOLD + _C.MAGENTA))
    if not report["constants"]:
        print("  (none)")
    else:
        for c in report["constants"]:
            name = _c(c["name"], _C.GREEN)
            ctype = _c(c["type"], _C.CYAN)
            rep = _c(c["repr"], _C.GRAY)
            print(f"  - {name} : <{ctype}> = {rep}")


# ---------- Package (library) explorer ----------
def list_modules(package_name: str, recursive: bool = True, include_private: bool = False) -> list[str]:
    """
    Return a list of importable modules/submodules under a package.

    Args:
        package_name: top-level package (e.g. "langgraph")
        recursive: walk subpackages recursively
        include_private: if False, skip dotted-path parts that start with "_"
    """
    pkg = importlib.import_module(package_name)

    # If it's a plain module, just return itself
    if not hasattr(pkg, "__path__"):
        return [package_name]

    results: list[str] = []
    for _, name, _ in pkgutil.walk_packages(pkg.__path__, prefix=f"{package_name}."):
        if not include_private:
            parts = name.split(".")
            if any(p.startswith("_") for p in parts):
                continue
        results.append(name)
    return sorted(results)


def explore_package(
    package_name: str,
    recursive: bool = True,
    include_private: bool = False,
    limit: int | None = None,
) -> None:
    """
    List modules in a package, then print a per-module report.

    Args:
        package_name: e.g. "langgraph"
        recursive: recurse into subpackages
        include_private: include modules whose path parts start with "_"
        limit: if set, limit how many modules to report (useful for very large packages)
    """
    modules = list_modules(package_name, recursive=recursive, include_private=include_private)

    print(_c(f"\n=== Modules in '{package_name}' ({'including' if include_private else 'excluding'} private) ===",
             _C.BOLD + _C.BLUE))
    for m in modules if (limit is None) else modules[:limit]:
        print("  -", m)

    print(_c(f"\n=== Detailed reports ===", _C.BOLD + _C.BLUE))
    targets = modules if (limit is None) else modules[:limit]
    for m in targets:
        try:
            report = explore_module(m)
            pretty_print_report(report)
        except Exception as e:
            print(_c(f"\n=== Module: {m} ===", _C.BOLD + _C.BLUE))
            print(_c("  (failed to import) ", _C.YELLOW) + repr(e))


# ------------------ Example usage in notebook ------------------
# First: list modules, then show a report for each (public only)
explore_package("langgraph", recursive=True, include_private=False)
# If the package is huge, you can limit output:
# explore_package("langgraph", recursive=True, include_private=False, limit=10)



=== Modules in 'langgraph' (excluding private) ===
  - langgraph.channels
  - langgraph.channels.any_value
  - langgraph.channels.base
  - langgraph.channels.binop
  - langgraph.channels.ephemeral_value
  - langgraph.channels.last_value
  - langgraph.channels.named_barrier_value
  - langgraph.channels.topic
  - langgraph.channels.untracked_value
  - langgraph.config
  - langgraph.constants
  - langgraph.errors
  - langgraph.func
  - langgraph.graph
  - langgraph.graph.message
  - langgraph.graph.state
  - langgraph.graph.ui
  - langgraph.managed
  - langgraph.managed.base
  - langgraph.managed.is_last_step
  - langgraph.prebuilt
  - langgraph.prebuilt.chat_agent_executor
  - langgraph.prebuilt.interrupt
  - langgraph.prebuilt.tool_node
  - langgraph.prebuilt.tool_validator
  - langgraph.pregel
  - langgraph.pregel.debug
  - langgraph.pregel.main
  - langgraph.pregel.protocol
  - langgraph.pregel.remote
  - langgraph.pregel.types
  - langgraph.runtime
  - langgraph.types
  - langgraph.

In [ ]:
import inspect
from langgraph.graph import StateGraph
print(inspect.signature(StateGraph.__init__))

(self, state_schema: 'type[StateT]', context_schema: 'type[ContextT] | None' = None, *, input_schema: 'type[InputT] | None' = None, output_schema: 'type[OutputT] | None' = None, **kwargs: 'Unpack[DeprecatedKwargs]') -> 'None'


In [ ]:
# === Auto-detect "class(class)" constructor params in a module ===
from __future__ import annotations
import importlib, inspect, typing as t
from typing import get_origin, get_args, get_type_hints

# Heuristic names that usually indicate a class/type param
_CLASSY_NAMES = {
    "cls", "klass", "class_", "impl", "implementation",
    "factory", "adapter", "strategy",
    "serializer_cls", "state_schema", "schema",
    "model", "model_cls", "type_", "node_type", "backend_cls"
}

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _public_classes_in_module(mod) -> dict[str, type]:
    out = {}
    for name in dir(mod):
        if not _is_public(name):
            continue
        obj = getattr(mod, name)
        if inspect.isclass(obj):
            out[name] = obj
    return out

def _annotation_wants_class(ann) -> bool:
    """True if annotation looks like type[T] / typing.Type[T] / raw type."""
    try:
        origin = get_origin(ann)
        if origin is type:  # e.g. type[T]
            return True
        txt = str(ann)
        if "typing.Type[" in txt or "type[" in txt:
            return True
    except Exception:
        pass
    return ann is type  # raw 'type'

def _annotation_base_type(ann):
    """Return T from type[T]/Type[T], else None."""
    try:
        origin = get_origin(ann)
        if origin is type:
            args = get_args(ann)
            if args and inspect.isclass(args[0]):
                return args[0]
    except Exception:
        pass
    return None

def _name_suggests_class(param_name: str) -> bool:
    if param_name in _CLASSY_NAMES:
        return True
    if param_name.endswith("_class") or param_name.endswith("_cls"):
        return True
    return False

def find_class_params(module_name: str) -> list[dict]:
    """
    For each public class in module, analyze __init__ params and
    guess which ones expect a CLASS. Suggest candidate classes.
    """
    mod = importlib.import_module(module_name)
    pub_classes = _public_classes_in_module(mod)

    reports = []
    for cls_name, cls in sorted(pub_classes.items()):
        try:
            sig = inspect.signature(cls.__init__)
        except Exception:
            continue

        # evaluated hints work best; fall back to raw annotations if needed
        try:
            hints = get_type_hints(cls.__init__, globalns=vars(mod), include_extras=True)
        except Exception:
            hints = {}

        params_info = []
        for pname, p in sig.parameters.items():
            if pname == "self":
                continue

            ann = hints.get(pname, p.annotation)
            wants_class = _annotation_wants_class(ann) or _name_suggests_class(pname)

            if wants_class:
                base = _annotation_base_type(ann)
                candidates = []
                if base and inspect.isclass(base):
                    # suggest classes in this module that are subclasses of the base
                    for cand_name, cand_cls in pub_classes.items():
                        try:
                            if issubclass(cand_cls, base) and cand_cls is not base:
                                candidates.append(cand_name)
                        except Exception:
                            pass
                else:
                    # unknown base -> soft suggestions: all public classes in this module
                    candidates = [n for n in pub_classes.keys() if n != cls_name]

                params_info.append({
                    "param": pname,
                    "annotation": str(ann),
                    "base_type": base.__name__ if isinstance(base, type) else None,
                    "candidates": sorted(candidates)
                })

        if params_info:
            reports.append({
                "class": f"{module_name}.{cls_name}",
                "init_signature": str(sig),
                "expects_class_params": params_info
            })

    return reports

def print_class_param_report(reports: list[dict]) -> None:
    if not reports:
        print("No constructor parameters that clearly expect a CLASS were found.")
        return
    for rep in reports:
        print(f"\n=== {rep['class']} ===")
        print(f"__init__ signature: {rep['init_signature']}")
        for p in rep["expects_class_params"]:
            print(f"  - param: {p['param']}")
            print(f"      annotation: {p['annotation']}")
            if p["base_type"]:
                print(f"      expects: CLASS (subclass of {p['base_type']})")
            else:
                print(f"      expects: CLASS (base type unclear; check docs/examples)")
            print(f"      candidate classes in module: {p['candidates']}")

# ---------------- Example: run on langgraph.graph ----------------
reports = find_class_params("langgraph.graph")
print_class_param_report(reports)



=== langgraph.graph.StateGraph ===
__init__ signature: (self, state_schema: 'type[StateT]', context_schema: 'type[ContextT] | None' = None, *, input_schema: 'type[InputT] | None' = None, output_schema: 'type[OutputT] | None' = None, **kwargs: 'Unpack[DeprecatedKwargs]') -> 'None'
  - param: state_schema
      annotation: type[StateT]
      expects: CLASS (base type unclear; check docs/examples)
      candidate classes in module: ['MessageGraph', 'MessagesState']
  - param: context_schema
      annotation: type[ContextT] | None
      expects: CLASS (base type unclear; check docs/examples)
      candidate classes in module: ['MessageGraph', 'MessagesState']
  - param: input_schema
      annotation: type[InputT] | None
      expects: CLASS (base type unclear; check docs/examples)
      candidate classes in module: ['MessageGraph', 'MessagesState']
  - param: output_schema
      annotation: type[OutputT] | None
      expects: CLASS (base type unclear; check docs/examples)
      candidate 

In [ ]:
from __future__ import annotations
import importlib, inspect, typing as t
from typing import get_origin, get_args, get_type_hints

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _public_classes(mod) -> dict[str, type]:
    out = {}
    for name in dir(mod):
        if _is_public(name):
            obj = getattr(mod, name)
            if inspect.isclass(obj):
                out[name] = obj
    return out

def _ann_wants_class(ann) -> bool:
    try:
        origin = get_origin(ann)
        if origin is type:           # e.g. type[T]
            return True
        txt = str(ann)
        if "typing.Type[" in txt or "type[" in txt:
            return True
    except Exception:
        pass
    return ann is type              # raw 'type'

def _ann_base_type(ann):
    try:
        if get_origin(ann) is type:
            args = get_args(ann)
            if args and inspect.isclass(args[0]):
                return args[0]
    except Exception:
        pass
    return None

def list_class_ctor_type_params(module_name: str) -> list[dict]:
    mod = importlib.import_module(module_name)
    classes = _public_classes(mod)
    reports = []

    for cls_name, cls in sorted(classes.items()):
        try:
            sig = inspect.signature(cls.__init__)
        except Exception:
            continue
        try:
            hints = get_type_hints(cls.__init__, globalns=vars(mod), include_extras=True)
        except Exception:
            hints = {}

        params = []
        for pname, p in sig.parameters.items():
            if pname == "self":
                continue
            ann = hints.get(pname, p.annotation)
            if _ann_wants_class(ann):
                base = _ann_base_type(ann)
                # suggest candidate classes from same module if base known
                candidates = []
                if base:
                    for cand_name, cand_cls in classes.items():
                        try:
                            if issubclass(cand_cls, base) and cand_cls is not base:
                                candidates.append(cand_name)
                        except Exception:
                            pass
                params.append({
                    "param": pname,
                    "annotation": str(ann),
                    "base_type": base.__name__ if isinstance(base, type) else None,
                    "candidates_in_module": sorted(candidates) if candidates else None
                })

        if params:
            reports.append({
                "class": f"{module_name}.{cls_name}",
                "init_signature": str(sig),
                "type_params": params
            })
    return reports

def pretty_print_type_param_report(module_name: str):
    reps = list_class_ctor_type_params(module_name)
    if not reps:
        print(f"(No classes in {module_name} with constructor params typed as type[T]/Type[T].)")
        return
    for r in reps:
        print(f"\n=== {r['class']} ===")
        print(f"__init__: {r['init_signature']}")
        for p in r["type_params"]:
            print(f"  - param: {p['param']}")
            print(f"      annotation: {p['annotation']}")
            print(f"      expects: CLASS (type)")
            if p['base_type']:
                print(f"      base type: {p['base_type']}")
            if p['candidates_in_module']:
                print(f"      candidates in module: {p['candidates_in_module']}")

# Example: run for langgraph.graph
pretty_print_type_param_report("langgraph.graph")


=== langgraph.graph.StateGraph ===
__init__: (self, state_schema: 'type[StateT]', context_schema: 'type[ContextT] | None' = None, *, input_schema: 'type[InputT] | None' = None, output_schema: 'type[OutputT] | None' = None, **kwargs: 'Unpack[DeprecatedKwargs]') -> 'None'
  - param: state_schema
      annotation: type[StateT]
      expects: CLASS (type)
  - param: context_schema
      annotation: type[ContextT] | None
      expects: CLASS (type)
  - param: input_schema
      annotation: type[InputT] | None
      expects: CLASS (type)
  - param: output_schema
      annotation: type[OutputT] | None
      expects: CLASS (type)


In [ ]:
# scan the whole package (public modules) for constructor params that expect classes
mods = list_modules("langgraph", recursive=True, include_private=False)

for m in mods:
    reps = list_class_ctor_type_params(m)  # from the helper we added earlier
    if not reps:
        continue
    print(f"\n### Module: {m}")
    for r in reps:
        print(f"  - {r['class']}")
        print(f"    __init__: {r['init_signature']}")
        for p in r["type_params"]:
            print(f"      param: {p['param']} | {p['annotation']}  → expects CLASS")
            if p["base_type"]:
                print(f"        base type: {p['base_type']}")
            if p["candidates_in_module"]:
                print(f"        candidates: {p['candidates_in_module']}")



### Module: langgraph.channels
  - langgraph.channels.BinaryOperatorAggregate
    __init__: (self, typ: type[~Value], operator: Callable[[~Value, ~Value], ~Value])
      param: typ | type[~Value]  → expects CLASS
  - langgraph.channels.NamedBarrierValue
    __init__: (self, typ: type[~Value], names: set[~Value]) -> None
      param: typ | type[~Value]  → expects CLASS
  - langgraph.channels.NamedBarrierValueAfterFinish
    __init__: (self, typ: type[~Value], names: set[~Value]) -> None
      param: typ | type[~Value]  → expects CLASS
  - langgraph.channels.Topic
    __init__: (self, typ: 'type[Value]', accumulate: 'bool' = False) -> 'None'
      param: typ | type[Value]  → expects CLASS
  - langgraph.channels.UntrackedValue
    __init__: (self, typ: 'type[Value]', guard: 'bool' = True) -> 'None'
      param: typ | type[Value]  → expects CLASS

### Module: langgraph.channels.binop
  - langgraph.channels.binop.BinaryOperatorAggregate
    __init__: (self, typ: type[~Value], operator: Cal

In [ ]:
# === Generate "ClassA(ClassB)" combos per module for any package ===
from __future__ import annotations
import importlib, inspect, pkgutil, typing as typing
from typing import get_origin, get_args, get_type_hints

# ---------- module listing ----------
def list_modules(package_name: str, recursive: bool = True, include_private: bool = False) -> list[str]:
    pkg = importlib.import_module(package_name)
    if not hasattr(pkg, "__path__"):
        return [package_name]
    out = []
    for _, name, _ in pkgutil.walk_packages(pkg.__path__, prefix=f"{package_name}."):
        if not include_private and any(part.startswith("_") for part in name.split(".")):
            continue
        out.append(name)
    return sorted(out)

# ---------- helpers ----------
SCHEMAY_NAMES = {"schema","state_schema","model","model_cls","cls","class_","*_cls","*_class"}
VALUE_TYPE_NAMES = {"typ","type","value_type"}

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _public_classes(mod) -> dict[str, type]:
    out = {}
    for n in dir(mod):
        if not _is_public(n):
            continue
        obj = getattr(mod, n)
        if inspect.isclass(obj):
            out[n] = obj
    return out

def _ann_wants_class(ann) -> bool:
    try:
        origin = get_origin(ann)
        if origin is type:
            return True
        s = str(ann)
        if "typing.Type[" in s or "type[" in s:
            return True
    except Exception:
        pass
    return ann is type

def _ann_base_type(ann):
    try:
        if get_origin(ann) is type:
            args = get_args(ann)
            if args and inspect.isclass(args[0]):
                return args[0]
    except Exception:
        pass
    return None

def _name_is_schemaish(pname: str) -> bool:
    if pname in SCHEMAY_NAMES:
        return True
    if pname.endswith("_schema") or pname.endswith("_cls") or pname.endswith("_class"):
        return True
    return False

def _name_is_value_typeish(pname: str) -> bool:
    return pname in VALUE_TYPE_NAMES

# ---------- core: build combos ----------
def combos_for_module(module_name: str) -> list[str]:
    """
    Return one-liner combos like 'StateGraph(MessagesState)' for a given module.
    """
    mod = importlib.import_module(module_name)
    classes = _public_classes(mod)
    combos: list[str] = []

    for cls_name, cls in sorted(classes.items()):
        # get __init__ signature & hints

        if "deprecated" in cls_name.lower():
          continue
        try:
            sig = inspect.signature(cls.__init__)
        except Exception:
            continue
        try:
            hints = get_type_hints(cls.__init__, globalns=vars(mod), include_extras=True)
        except Exception:
            hints = {}

        for pname, param in sig.parameters.items():
            if pname == "self":
                continue
            ann = hints.get(pname, param.annotation)
            if not _ann_wants_class(ann):
                continue

            # If base type is concrete, suggest subclasses in same module
            base = _ann_base_type(ann)
            if base and inspect.isclass(base):
                candidates = []
                for other_name, other_cls in classes.items():
                    try:
                        if other_cls is not base and issubclass(other_cls, base):
                            candidates.append(other_name)
                    except Exception:
                        pass
                for cand in sorted(set(candidates)):
                    combos.append(f"{cls_name}({cand})")
                # If we found concrete candidates, continue; otherwise fall through to heuristics
                if candidates:
                    continue

            # Heuristic 1: schema-like param names -> prefer classes with State/Model/Schema in name
            if _name_is_schemaish(pname):
                schemaish = [n for n in classes.keys() if any(tok in n.lower() for tok in ("state","model","schema")) and n != cls_name]
                for cand in sorted(set(schemaish)):
                    combos.append(f"{cls_name}({cand})")
                if schemaish:
                    continue

            # Heuristic 2: value-type params ('typ'/'type') -> suggest builtins
            if _name_is_value_typeish(pname):
                for builtin in ("int","str","dict"):
                    combos.append(f"{cls_name}({builtin})")
                continue

            # Fallback: if we got here, we still know it's a class; offer all other classes in module
            for cand in sorted(n for n in classes.keys() if n != cls_name):
                combos.append(f"{cls_name}({cand})")

    # Deduplicate while preserving order
    seen = set()
    uniq = []
    for c in combos:
        if c not in seen:
            seen.add(c)
            uniq.append(c)
    return uniq

def combos_for_package(package_name: str, recursive: bool = True, include_private: bool = False):
    """
    Print combos per module in a package.
    """
    mods = list_modules(package_name, recursive=recursive, include_private=include_private)
    print(f"=== Modules in '{package_name}' ===")
    for m in mods:
        print("  -", m)
    for m in mods:
        lines = combos_for_module(m)
        if not lines:
            continue
        print(f"\n### {m}")
        for line in lines:
            print(" -", line)

# --------- Example: run for langgraph, public modules only ---------
# Only this call is needed:
combos_for_package("langgraph", recursive=True, include_private=False)

# If you only want a single module:
# for line in combos_for_module("langgraph.graph"):
#     print(line)


=== Modules in 'langgraph' ===
  - langgraph.channels
  - langgraph.channels.any_value
  - langgraph.channels.base
  - langgraph.channels.binop
  - langgraph.channels.ephemeral_value
  - langgraph.channels.last_value
  - langgraph.channels.named_barrier_value
  - langgraph.channels.topic
  - langgraph.channels.untracked_value
  - langgraph.config
  - langgraph.constants
  - langgraph.errors
  - langgraph.func
  - langgraph.graph
  - langgraph.graph.message
  - langgraph.graph.state
  - langgraph.graph.ui
  - langgraph.managed
  - langgraph.managed.base
  - langgraph.managed.is_last_step
  - langgraph.prebuilt
  - langgraph.prebuilt.chat_agent_executor
  - langgraph.prebuilt.interrupt
  - langgraph.prebuilt.tool_node
  - langgraph.prebuilt.tool_validator
  - langgraph.pregel
  - langgraph.pregel.debug
  - langgraph.pregel.main
  - langgraph.pregel.protocol
  - langgraph.pregel.remote
  - langgraph.pregel.types
  - langgraph.runtime
  - langgraph.types
  - langgraph.typing
  - langgraph.

In [4]:
import importlib, inspect, pkgutil, sys
from types import ModuleType

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _load_module(name: str) -> ModuleType | None:
    try:
        return importlib.import_module(name)
    except Exception:
        return None

def _iter_submodules(pkg_name: str):
    pkg = _load_module(pkg_name)
    if not pkg or not hasattr(pkg, "__path__"):
        return
    yield pkg_name, pkg
    for m in pkgutil.walk_packages(pkg.__path__, prefix=pkg_name + "."):
        mod = _load_module(m.name)
        if mod is not None:
            yield m.name, mod

def _public_names(mod: ModuleType):
    if hasattr(mod, "__all__"):
        return [n for n in mod.__all__ if _is_public(n)]
    return [n for n in dir(mod) if _is_public(n)]

def _classify(mod: ModuleType):
    classes, funcs, consts = [], [], []
    for n in _public_names(mod):
        try:
            obj = getattr(mod, n)
        except Exception:
            continue
        if inspect.isclass(obj) and getattr(obj, "__module__", None) == mod.__name__:
            classes.append(n)
        elif inspect.isfunction(obj) and getattr(obj, "__module__", None) == mod.__name__:
            funcs.append(n)
        elif not inspect.ismodule(obj) and not inspect.isroutine(obj) and not inspect.isclass(obj):
            consts.append(n)
    return sorted(classes), sorted(funcs), sorted(consts)

def dump_api(
    root: str = "langgraph",
    as_markdown: bool = True,
    include_const_imports: bool = False,
) -> str:
    """
    Return a formatted dump (string) of module -> Classes/Functions/Constants and import lines.
    Use in Jupyter: print(...) or display(Markdown(...)).
    """
    lines = []
    for modname, mod in _iter_submodules(root):
        classes, funcs, consts = _classify(mod)
        if not (classes or funcs or consts):
            continue

        # Header
        if as_markdown:
            lines.append(f"## Module: `{modname}`\n")
        else:
            lines.append(f"=== Module: {modname} ===\n")

        # Classes
        if classes:
            lines.append("**Classes:**\n" if as_markdown else "Classes:\n")
            for c in classes:
                lines.append(f"- {c}")
            lines.append("")  # blank line

        # Functions
        if funcs:
            lines.append("**Functions:**\n" if as_markdown else "Functions:\n")
            for f in funcs:
                lines.append(f"- {f}")
            lines.append("")

        # Constants / Objects
        if consts:
            lines.append("**Constants / Objects:**\n" if as_markdown else "Constants / Objects:\n")
            for k in consts:
                lines.append(f"- {k}")
            lines.append("")

        # Import statements
        imports = []
        imports += [f"from {modname} import {c}" for c in classes]
        imports += [f"from {modname} import {f}" for f in funcs]
        if include_const_imports:
            imports += [f"from {modname} import {k}" for k in consts]

        if imports:
            lines.append("**Import statements:**\n" if as_markdown else "Import statements:\n")
            for line in imports:
                lines.append(f"- `{line}`" if as_markdown else f"  {line}")
            lines.append("")

        lines.append("")  # spacer between modules

    return "\n".join(lines).strip()


In [5]:
# 1) Dump everything under `langgraph` (Markdown output)
text = dump_api("langgraph", as_markdown=True)
from IPython.display import Markdown, display
display(Markdown(text))  # nicely formatted in the notebook

# 2) Focus on a subpackage (e.g., prebuilt)
text_prebuilt = dump_api("langgraph.prebuilt", as_markdown=True)
display(Markdown(text_prebuilt))

# 3) Save to a file (e.g., for your repo)
with open("langgraph_api.md", "w", encoding="utf-8") as f:
    f.write(text)


## Module: `langgraph._internal._cache`

**Functions:**

- default_cache_key

**Constants / Objects:**

- annotations

**Import statements:**

- `from langgraph._internal._cache import default_cache_key`


## Module: `langgraph._internal._config`

**Functions:**

- ensure_config
- get_async_callback_manager_for_config
- get_callback_manager_for_config
- merge_configs
- patch_checkpoint_map
- patch_config
- patch_configurable
- recast_checkpoint_ns

**Constants / Objects:**

- CONF
- CONFIG_KEYS
- CONFIG_KEY_CHECKPOINT_ID
- CONFIG_KEY_CHECKPOINT_MAP
- CONFIG_KEY_CHECKPOINT_NS
- COPIABLE_KEYS
- Callbacks
- DEFAULT_RECURSION_LIMIT
- NS_END
- NS_SEP
- annotations
- var_child_runnable_config

**Import statements:**

- `from langgraph._internal._config import ensure_config`
- `from langgraph._internal._config import get_async_callback_manager_for_config`
- `from langgraph._internal._config import get_callback_manager_for_config`
- `from langgraph._internal._config import merge_configs`
- `from langgraph._internal._config import patch_checkpoint_map`
- `from langgraph._internal._config import patch_config`
- `from langgraph._internal._config import patch_configurable`
- `from langgraph._internal._config import recast_checkpoint_ns`


## Module: `langgraph._internal._constants`

**Constants / Objects:**

- CACHE_NS_WRITES
- CONF
- CONFIG_KEY_CACHE
- CONFIG_KEY_CALL
- CONFIG_KEY_CHECKPOINTER
- CONFIG_KEY_CHECKPOINT_ID
- CONFIG_KEY_CHECKPOINT_MAP
- CONFIG_KEY_CHECKPOINT_NS
- CONFIG_KEY_DURABILITY
- CONFIG_KEY_NODE_FINISHED
- CONFIG_KEY_READ
- CONFIG_KEY_RESUME_MAP
- CONFIG_KEY_RESUMING
- CONFIG_KEY_RUNNER_SUBMIT
- CONFIG_KEY_RUNTIME
- CONFIG_KEY_SCRATCHPAD
- CONFIG_KEY_SEND
- CONFIG_KEY_STREAM
- CONFIG_KEY_TASK_ID
- CONFIG_KEY_THREAD_ID
- ERROR
- INPUT
- INTERRUPT
- Literal
- NO_WRITES
- NS_END
- NS_SEP
- NULL_TASK_ID
- PREVIOUS
- PULL
- PUSH
- RESERVED
- RESUME
- RETURN
- TASKS


## Module: `langgraph._internal._fields`

**Functions:**

- get_cached_annotated_keys
- get_enhanced_type_hints
- get_field_default
- get_update_as_tuples

**Constants / Objects:**

- ANNOTATED_KEYS_CACHE
- MISSING
- NotRequired
- Optional
- ReadOnly
- Required
- Union
- annotations

**Import statements:**

- `from langgraph._internal._fields import get_cached_annotated_keys`
- `from langgraph._internal._fields import get_enhanced_type_hints`
- `from langgraph._internal._fields import get_field_default`
- `from langgraph._internal._fields import get_update_as_tuples`


## Module: `langgraph._internal._future`

**Functions:**

- chain_future
- run_coroutine_threadsafe

**Constants / Objects:**

- AnyFuture
- CONTEXT_NOT_SUPPORTED
- EAGER_NOT_SUPPORTED
- T
- Union
- annotations

**Import statements:**

- `from langgraph._internal._future import chain_future`
- `from langgraph._internal._future import run_coroutine_threadsafe`


## Module: `langgraph._internal._pydantic`

**Functions:**

- create_model
- get_fields
- is_supported_by_pydantic

**Constants / Objects:**

- DEFAULT_REF_TEMPLATE
- JsonSchemaMode
- NO_DEFAULT
- TypedDict
- annotations

**Import statements:**

- `from langgraph._internal._pydantic import create_model`
- `from langgraph._internal._pydantic import get_fields`
- `from langgraph._internal._pydantic import is_supported_by_pydantic`


## Module: `langgraph._internal._queue`

**Classes:**

- AsyncQueue
- Semaphore
- SyncQueue

**Constants / Objects:**

- PY_310
- annotations

**Import statements:**

- `from langgraph._internal._queue import AsyncQueue`
- `from langgraph._internal._queue import Semaphore`
- `from langgraph._internal._queue import SyncQueue`


## Module: `langgraph._internal._retry`

**Functions:**

- default_retry_on

**Import statements:**

- `from langgraph._internal._retry import default_retry_on`


## Module: `langgraph._internal._runnable`

**Classes:**

- RunnableCallable
- RunnableSeq
- StrEnum

**Functions:**

- coerce_to_runnable
- is_async_callable
- is_async_generator
- set_config_context

**Constants / Objects:**

- ANY_TYPE
- ASYNCIO_ACCEPTS_CONTEXT
- CONF
- CONFIG_KEY_RUNTIME
- Callable
- Input
- KWARGS_CONFIG_KEYS
- LCRunnableLike
- MISSING
- Optional
- Output
- RunnableLike
- StreamWriter
- TypeGuard
- Union
- VALID_KINDS
- annotations
- var_child_runnable_config

**Import statements:**

- `from langgraph._internal._runnable import RunnableCallable`
- `from langgraph._internal._runnable import RunnableSeq`
- `from langgraph._internal._runnable import StrEnum`
- `from langgraph._internal._runnable import coerce_to_runnable`
- `from langgraph._internal._runnable import is_async_callable`
- `from langgraph._internal._runnable import is_async_generator`
- `from langgraph._internal._runnable import set_config_context`


## Module: `langgraph._internal._scratchpad`

**Classes:**

- PregelScratchpad

**Constants / Objects:**

- Callable

**Import statements:**

- `from langgraph._internal._scratchpad import PregelScratchpad`


## Module: `langgraph._internal._typing`

**Classes:**

- DataclassLike
- DeprecatedKwargs
- TypedDictLikeV1
- TypedDictLikeV2

**Constants / Objects:**

- ClassVar
- EMPTY_SEQ
- MISSING
- StateLike
- TypeAlias
- TypedDict
- Union
- annotations

**Import statements:**

- `from langgraph._internal._typing import DataclassLike`
- `from langgraph._internal._typing import DeprecatedKwargs`
- `from langgraph._internal._typing import TypedDictLikeV1`
- `from langgraph._internal._typing import TypedDictLikeV2`


## Module: `langgraph.channels.any_value`

**Classes:**

- AnyValue

**Import statements:**

- `from langgraph.channels.any_value import AnyValue`


## Module: `langgraph.channels.base`

**Classes:**

- BaseChannel

**Import statements:**

- `from langgraph.channels.base import BaseChannel`


## Module: `langgraph.channels.binop`

**Classes:**

- BinaryOperatorAggregate

**Import statements:**

- `from langgraph.channels.binop import BinaryOperatorAggregate`


## Module: `langgraph.channels.ephemeral_value`

**Classes:**

- EphemeralValue

**Import statements:**

- `from langgraph.channels.ephemeral_value import EphemeralValue`


## Module: `langgraph.channels.last_value`

**Classes:**

- LastValue
- LastValueAfterFinish

**Import statements:**

- `from langgraph.channels.last_value import LastValue`
- `from langgraph.channels.last_value import LastValueAfterFinish`


## Module: `langgraph.channels.named_barrier_value`

**Classes:**

- NamedBarrierValue
- NamedBarrierValueAfterFinish

**Import statements:**

- `from langgraph.channels.named_barrier_value import NamedBarrierValue`
- `from langgraph.channels.named_barrier_value import NamedBarrierValueAfterFinish`


## Module: `langgraph.channels.topic`

**Classes:**

- Topic

**Import statements:**

- `from langgraph.channels.topic import Topic`


## Module: `langgraph.channels.untracked_value`

**Classes:**

- UntrackedValue

**Import statements:**

- `from langgraph.channels.untracked_value import UntrackedValue`


## Module: `langgraph.config`

**Functions:**

- get_config
- get_store
- get_stream_writer

**Constants / Objects:**

- CONF
- CONFIG_KEY_RUNTIME
- StreamWriter
- var_child_runnable_config

**Import statements:**

- `from langgraph.config import get_config`
- `from langgraph.config import get_store`
- `from langgraph.config import get_stream_writer`


## Module: `langgraph.constants`

**Constants / Objects:**

- CONF
- CONFIG_KEY_CHECKPOINTER
- END
- START
- TAG_HIDDEN
- TAG_NOSTREAM
- TASKS


## Module: `langgraph.errors`

**Classes:**

- EmptyInputError
- ErrorCode
- GraphBubbleUp
- GraphInterrupt
- GraphRecursionError
- InvalidUpdateError
- NodeInterrupt
- ParentCommand
- TaskNotFound

**Import statements:**

- `from langgraph.errors import EmptyInputError`
- `from langgraph.errors import ErrorCode`
- `from langgraph.errors import GraphBubbleUp`
- `from langgraph.errors import GraphInterrupt`
- `from langgraph.errors import GraphRecursionError`
- `from langgraph.errors import InvalidUpdateError`
- `from langgraph.errors import NodeInterrupt`
- `from langgraph.errors import ParentCommand`
- `from langgraph.errors import TaskNotFound`


## Module: `langgraph.func`

**Classes:**

- entrypoint

**Functions:**

- task

**Import statements:**

- `from langgraph.func import entrypoint`
- `from langgraph.func import task`


## Module: `langgraph.graph`

**Constants / Objects:**

- END
- START


## Module: `langgraph.graph._branch`

**Classes:**

- BranchSpec

**Constants / Objects:**

- Callable
- END
- Literal
- PASSTHROUGH
- START
- Union
- annotations

**Import statements:**

- `from langgraph.graph._branch import BranchSpec`


## Module: `langgraph.graph._node`

**Classes:**

- StateNodeSpec

**Constants / Objects:**

- ContextT
- EMPTY_SEQ
- NodeInputT
- NodeInputT_contra
- StateNode
- StreamWriter
- TypeAlias
- Union
- annotations

**Import statements:**

- `from langgraph.graph._node import StateNodeSpec`


## Module: `langgraph.graph.message`

**Classes:**

- MessageGraph
- MessagesState

**Functions:**

- add_messages

**Import statements:**

- `from langgraph.graph.message import MessageGraph`
- `from langgraph.graph.message import MessagesState`
- `from langgraph.graph.message import add_messages`


## Module: `langgraph.graph.state`

**Classes:**

- CompiledStateGraph
- StateGraph

**Import statements:**

- `from langgraph.graph.state import CompiledStateGraph`
- `from langgraph.graph.state import StateGraph`


## Module: `langgraph.graph.ui`

**Classes:**

- RemoveUIMessage
- UIMessage

**Functions:**

- delete_ui_message
- push_ui_message
- ui_message_reducer

**Constants / Objects:**

- AnyUIMessage

**Import statements:**

- `from langgraph.graph.ui import RemoveUIMessage`
- `from langgraph.graph.ui import UIMessage`
- `from langgraph.graph.ui import delete_ui_message`
- `from langgraph.graph.ui import push_ui_message`
- `from langgraph.graph.ui import ui_message_reducer`


## Module: `langgraph.managed`

**Constants / Objects:**

- IsLastStep
- RemainingSteps


## Module: `langgraph.managed.base`

**Constants / Objects:**

- ManagedValueMapping
- ManagedValueSpec


## Module: `langgraph.managed.is_last_step`

**Classes:**

- RemainingStepsManager

**Constants / Objects:**

- IsLastStep

**Import statements:**

- `from langgraph.managed.is_last_step import RemainingStepsManager`


## Module: `langgraph.prebuilt.chat_agent_executor`

**Classes:**

- AgentState
- AgentStatePydantic
- AgentStateWithStructuredResponse
- AgentStateWithStructuredResponsePydantic

**Functions:**

- create_react_agent
- create_tool_calling_executor

**Import statements:**

- `from langgraph.prebuilt.chat_agent_executor import AgentState`
- `from langgraph.prebuilt.chat_agent_executor import AgentStatePydantic`
- `from langgraph.prebuilt.chat_agent_executor import AgentStateWithStructuredResponse`
- `from langgraph.prebuilt.chat_agent_executor import AgentStateWithStructuredResponsePydantic`
- `from langgraph.prebuilt.chat_agent_executor import create_react_agent`
- `from langgraph.prebuilt.chat_agent_executor import create_tool_calling_executor`


## Module: `langgraph.prebuilt.interrupt`

**Classes:**

- ActionRequest
- HumanInterrupt
- HumanInterruptConfig
- HumanResponse

**Constants / Objects:**

- Literal
- Optional
- TypedDict
- Union

**Import statements:**

- `from langgraph.prebuilt.interrupt import ActionRequest`
- `from langgraph.prebuilt.interrupt import HumanInterrupt`
- `from langgraph.prebuilt.interrupt import HumanInterruptConfig`
- `from langgraph.prebuilt.interrupt import HumanResponse`


## Module: `langgraph.prebuilt.tool_node`

**Classes:**

- InjectedState
- InjectedStore
- ToolNode

**Functions:**

- msg_content_output
- tools_condition

**Constants / Objects:**

- AnyMessage
- Callable
- INVALID_TOOL_NAME_ERROR_TEMPLATE
- Literal
- Optional
- REMOVE_ALL_MESSAGES
- Sequence
- TOOL_CALL_ERROR_TEMPLATE
- TOOL_MESSAGE_BLOCK_TYPES
- Tuple
- Type
- Union

**Import statements:**

- `from langgraph.prebuilt.tool_node import InjectedState`
- `from langgraph.prebuilt.tool_node import InjectedStore`
- `from langgraph.prebuilt.tool_node import ToolNode`
- `from langgraph.prebuilt.tool_node import msg_content_output`
- `from langgraph.prebuilt.tool_node import tools_condition`


## Module: `langgraph.prebuilt.tool_validator`

**Classes:**

- ValidationNode

**Constants / Objects:**

- AnyMessage
- Callable
- Dict
- Optional
- Sequence
- Tuple
- Type
- Union

**Import statements:**

- `from langgraph.prebuilt.tool_validator import ValidationNode`


## Module: `langgraph.pregel._algo`

**Classes:**

- Call
- LazyAtomicCounter
- PregelTaskWrites
- WritesProtocol

**Functions:**

- apply_writes
- checkpoint_null_version
- increment
- local_read
- prepare_next_tasks
- prepare_single_task
- should_interrupt
- task_path_str

**Constants / Objects:**

- All
- CACHE_NS_WRITES
- CONF
- CONFIG_KEY_CHECKPOINTER
- CONFIG_KEY_CHECKPOINT_ID
- CONFIG_KEY_CHECKPOINT_MAP
- CONFIG_KEY_CHECKPOINT_NS
- CONFIG_KEY_READ
- CONFIG_KEY_RESUME_MAP
- CONFIG_KEY_RUNTIME
- CONFIG_KEY_SCRATCHPAD
- CONFIG_KEY_SEND
- CONFIG_KEY_TASK_ID
- Callable
- Callbacks
- ChannelVersions
- DEFAULT_RUNTIME
- EMPTY_SEQ
- ERROR
- GetNextVersion
- INPUT_CACHE_KEY_TYPE
- INTERRUPT
- LAZY_ATOMIC_COUNTER_LOCK
- Literal
- MISSING
- ManagedValueMapping
- NO_WRITES
- NS_END
- NS_SEP
- NULL_TASK_ID
- Optional
- PREVIOUS
- PULL
- PUSH
- PUSH_TRIGGER
- PendingWrite
- RESERVED
- RESUME
- RETURN
- SUPPORTS_EXC_NOTES
- TAG_HIDDEN
- TASKS
- V
- annotations
- logger

**Import statements:**

- `from langgraph.pregel._algo import Call`
- `from langgraph.pregel._algo import LazyAtomicCounter`
- `from langgraph.pregel._algo import PregelTaskWrites`
- `from langgraph.pregel._algo import WritesProtocol`
- `from langgraph.pregel._algo import apply_writes`
- `from langgraph.pregel._algo import checkpoint_null_version`
- `from langgraph.pregel._algo import increment`
- `from langgraph.pregel._algo import local_read`
- `from langgraph.pregel._algo import prepare_next_tasks`
- `from langgraph.pregel._algo import prepare_single_task`
- `from langgraph.pregel._algo import should_interrupt`
- `from langgraph.pregel._algo import task_path_str`


## Module: `langgraph.pregel._call`

**Classes:**

- SyncAsyncFuture

**Functions:**

- call
- get_runnable_for_entrypoint
- get_runnable_for_task
- identifier

**Constants / Objects:**

- CACHE
- CONF
- CONFIG_KEY_CALL
- Callable
- P
- P1
- RETURN
- T
- annotations

**Import statements:**

- `from langgraph.pregel._call import SyncAsyncFuture`
- `from langgraph.pregel._call import call`
- `from langgraph.pregel._call import get_runnable_for_entrypoint`
- `from langgraph.pregel._call import get_runnable_for_task`
- `from langgraph.pregel._call import identifier`


## Module: `langgraph.pregel._checkpoint`

**Functions:**

- channels_from_checkpoint
- copy_checkpoint
- create_checkpoint
- empty_checkpoint

**Constants / Objects:**

- LATEST_VERSION
- MISSING
- ManagedValueMapping
- ManagedValueSpec
- annotations

**Import statements:**

- `from langgraph.pregel._checkpoint import channels_from_checkpoint`
- `from langgraph.pregel._checkpoint import copy_checkpoint`
- `from langgraph.pregel._checkpoint import create_checkpoint`
- `from langgraph.pregel._checkpoint import empty_checkpoint`


## Module: `langgraph.pregel._draw`

**Functions:**

- add_edge
- draw_graph

**Constants / Objects:**

- All
- CONF
- CONFIG_KEY_SEND
- Checkpointer
- END
- INPUT
- ManagedValueSpec
- START
- annotations

**Import statements:**

- `from langgraph.pregel._draw import add_edge`
- `from langgraph.pregel._draw import draw_graph`


## Module: `langgraph.pregel._executor`

**Classes:**

- AsyncBackgroundExecutor
- BackgroundExecutor
- Submit

**Functions:**

- gated
- next_tick

**Constants / Objects:**

- CONTEXT_NOT_SUPPORTED
- Callable
- P
- T
- annotations

**Import statements:**

- `from langgraph.pregel._executor import AsyncBackgroundExecutor`
- `from langgraph.pregel._executor import BackgroundExecutor`
- `from langgraph.pregel._executor import Submit`
- `from langgraph.pregel._executor import gated`
- `from langgraph.pregel._executor import next_tick`


## Module: `langgraph.pregel._io`

**Functions:**

- map_command
- map_input
- map_output_updates
- map_output_values
- read_channel
- read_channels

**Constants / Objects:**

- EMPTY_SEQ
- ERROR
- INTERRUPT
- Literal
- MISSING
- NULL_TASK_ID
- RESUME
- RETURN
- START
- TAG_HIDDEN
- TASKS
- annotations
- logger

**Import statements:**

- `from langgraph.pregel._io import map_command`
- `from langgraph.pregel._io import map_input`
- `from langgraph.pregel._io import map_output_updates`
- `from langgraph.pregel._io import map_output_values`
- `from langgraph.pregel._io import read_channel`
- `from langgraph.pregel._io import read_channels`


## Module: `langgraph.pregel._log`

**Constants / Objects:**

- logger


## Module: `langgraph.pregel._loop`

**Classes:**

- AsyncPregelLoop
- PregelLoop
- SyncPregelLoop

**Functions:**

- DuplexStream

**Constants / Objects:**

- All
- CONF
- CONFIG_KEY_CHECKPOINT_ID
- CONFIG_KEY_CHECKPOINT_MAP
- CONFIG_KEY_CHECKPOINT_NS
- CONFIG_KEY_RESUME_MAP
- CONFIG_KEY_RESUMING
- CONFIG_KEY_SCRATCHPAD
- CONFIG_KEY_STREAM
- CONFIG_KEY_TASK_ID
- CONFIG_KEY_THREAD_ID
- Callable
- ChannelVersions
- Durability
- EMPTY_SEQ
- ERROR
- GetNextVersion
- INPUT
- INTERRUPT
- Literal
- MISSING
- ManagedValueMapping
- ManagedValueSpec
- NS_END
- NS_SEP
- NULL_TASK_ID
- Optional
- P
- PUSH
- PendingWrite
- RESUME
- Self
- StreamChunk
- StreamMode
- TAG_HIDDEN
- V
- WRITES_IDX_MAP
- WritesT
- annotations

**Import statements:**

- `from langgraph.pregel._loop import AsyncPregelLoop`
- `from langgraph.pregel._loop import PregelLoop`
- `from langgraph.pregel._loop import SyncPregelLoop`
- `from langgraph.pregel._loop import DuplexStream`


## Module: `langgraph.pregel._messages`

**Classes:**

- StreamMessagesHandler

**Constants / Objects:**

- Callable
- Meta
- NS_SEP
- StreamChunk
- T
- TAG_HIDDEN
- TAG_NOSTREAM
- annotations

**Import statements:**

- `from langgraph.pregel._messages import StreamMessagesHandler`


## Module: `langgraph.pregel._read`

**Classes:**

- ChannelRead
- PregelNode

**Constants / Objects:**

- CONF
- CONFIG_KEY_READ
- Callable
- DEFAULT_BOUND
- INPUT_CACHE_KEY_TYPE
- READ_TYPE
- Union
- annotations

**Import statements:**

- `from langgraph.pregel._read import ChannelRead`
- `from langgraph.pregel._read import PregelNode`


## Module: `langgraph.pregel._retry`

**Functions:**

- arun_with_retry
- run_with_retry

**Constants / Objects:**

- CONF
- CONFIG_KEY_CHECKPOINT_NS
- CONFIG_KEY_RESUMING
- Callable
- NS_SEP
- SUPPORTS_EXC_NOTES
- annotations
- logger

**Import statements:**

- `from langgraph.pregel._retry import arun_with_retry`
- `from langgraph.pregel._retry import run_with_retry`


## Module: `langgraph.pregel._runner`

**Classes:**

- FuturesDict
- PregelRunner

**Constants / Objects:**

- CONF
- CONFIG_KEY_CALL
- CONFIG_KEY_SCRATCHPAD
- Callable
- Callbacks
- E
- ERROR
- EXCLUDED_FRAME_FNAMES
- F
- INTERRUPT
- MISSING
- NO_WRITES
- Optional
- RESUME
- RETURN
- SKIP_RERAISE_SET
- TAG_HIDDEN
- Union
- annotations

**Import statements:**

- `from langgraph.pregel._runner import FuturesDict`
- `from langgraph.pregel._runner import PregelRunner`


## Module: `langgraph.pregel._utils`

**Classes:**

- FunctionNonLocals
- NonLocals

**Functions:**

- find_subgraph_pregel
- get_function_nonlocals
- get_new_channel_versions
- is_xxh3_128_hexdigest

**Constants / Objects:**

- Callable
- ChannelVersions
- annotations

**Import statements:**

- `from langgraph.pregel._utils import FunctionNonLocals`
- `from langgraph.pregel._utils import NonLocals`
- `from langgraph.pregel._utils import find_subgraph_pregel`
- `from langgraph.pregel._utils import get_function_nonlocals`
- `from langgraph.pregel._utils import get_new_channel_versions`
- `from langgraph.pregel._utils import is_xxh3_128_hexdigest`


## Module: `langgraph.pregel._validate`

**Functions:**

- validate_graph
- validate_keys

**Constants / Objects:**

- All
- ManagedValueMapping
- RESERVED
- annotations

**Import statements:**

- `from langgraph.pregel._validate import validate_graph`
- `from langgraph.pregel._validate import validate_keys`


## Module: `langgraph.pregel._write`

**Classes:**

- ChannelWrite
- ChannelWriteEntry
- ChannelWriteTupleEntry

**Constants / Objects:**

- CONF
- CONFIG_KEY_SEND
- Callable
- MISSING
- Optional
- PASSTHROUGH
- R
- SKIP_WRITE
- TASKS
- TYPE_SEND
- Union
- annotations

**Import statements:**

- `from langgraph.pregel._write import ChannelWrite`
- `from langgraph.pregel._write import ChannelWriteEntry`
- `from langgraph.pregel._write import ChannelWriteTupleEntry`


## Module: `langgraph.pregel.debug`

**Classes:**

- CheckpointPayload
- CheckpointTask
- TaskPayload
- TaskResultPayload

**Import statements:**

- `from langgraph.pregel.debug import CheckpointPayload`
- `from langgraph.pregel.debug import CheckpointTask`
- `from langgraph.pregel.debug import TaskPayload`
- `from langgraph.pregel.debug import TaskResultPayload`


## Module: `langgraph.pregel.main`

**Classes:**

- NodeBuilder
- Pregel

**Import statements:**

- `from langgraph.pregel.main import NodeBuilder`
- `from langgraph.pregel.main import Pregel`


## Module: `langgraph.pregel.protocol`

**Classes:**

- PregelProtocol
- StreamProtocol

**Import statements:**

- `from langgraph.pregel.protocol import PregelProtocol`
- `from langgraph.pregel.protocol import StreamProtocol`


## Module: `langgraph.pregel.remote`

**Classes:**

- RemoteException
- RemoteGraph

**Import statements:**

- `from langgraph.pregel.remote import RemoteException`
- `from langgraph.pregel.remote import RemoteGraph`


## Module: `langgraph.pregel.types`

**Constants / Objects:**

- All
- StreamMode
- StreamWriter


## Module: `langgraph.runtime`

**Classes:**

- Runtime

**Functions:**

- get_runtime

**Import statements:**

- `from langgraph.runtime import Runtime`
- `from langgraph.runtime import get_runtime`


## Module: `langgraph.types`

**Classes:**

- CachePolicy
- Command
- Interrupt
- PregelExecutableTask
- PregelTask
- RetryPolicy
- Send
- StateSnapshot
- StateUpdate

**Functions:**

- interrupt

**Constants / Objects:**

- All
- Checkpointer
- Durability
- StreamMode
- StreamWriter

**Import statements:**

- `from langgraph.types import CachePolicy`
- `from langgraph.types import Command`
- `from langgraph.types import Interrupt`
- `from langgraph.types import PregelExecutableTask`
- `from langgraph.types import PregelTask`
- `from langgraph.types import RetryPolicy`
- `from langgraph.types import Send`
- `from langgraph.types import StateSnapshot`
- `from langgraph.types import StateUpdate`
- `from langgraph.types import interrupt`


## Module: `langgraph.typing`

**Constants / Objects:**

- ContextT
- InputT
- OutputT
- StateT
- StateT_co
- StateT_contra


## Module: `langgraph.utils.runnable`

**Constants / Objects:**

- RunnableLike


## Module: `langgraph.warnings`

**Classes:**

- LangGraphDeprecatedSinceV05
- LangGraphDeprecatedSinceV10
- LangGraphDeprecationWarning

**Import statements:**

- `from langgraph.warnings import LangGraphDeprecatedSinceV05`
- `from langgraph.warnings import LangGraphDeprecatedSinceV10`
- `from langgraph.warnings import LangGraphDeprecationWarning`

## Module: `langgraph.prebuilt.chat_agent_executor`

**Classes:**

- AgentState
- AgentStatePydantic
- AgentStateWithStructuredResponse
- AgentStateWithStructuredResponsePydantic

**Functions:**

- create_react_agent
- create_tool_calling_executor

**Import statements:**

- `from langgraph.prebuilt.chat_agent_executor import AgentState`
- `from langgraph.prebuilt.chat_agent_executor import AgentStatePydantic`
- `from langgraph.prebuilt.chat_agent_executor import AgentStateWithStructuredResponse`
- `from langgraph.prebuilt.chat_agent_executor import AgentStateWithStructuredResponsePydantic`
- `from langgraph.prebuilt.chat_agent_executor import create_react_agent`
- `from langgraph.prebuilt.chat_agent_executor import create_tool_calling_executor`


## Module: `langgraph.prebuilt.interrupt`

**Classes:**

- ActionRequest
- HumanInterrupt
- HumanInterruptConfig
- HumanResponse

**Constants / Objects:**

- Literal
- Optional
- TypedDict
- Union

**Import statements:**

- `from langgraph.prebuilt.interrupt import ActionRequest`
- `from langgraph.prebuilt.interrupt import HumanInterrupt`
- `from langgraph.prebuilt.interrupt import HumanInterruptConfig`
- `from langgraph.prebuilt.interrupt import HumanResponse`


## Module: `langgraph.prebuilt.tool_node`

**Classes:**

- InjectedState
- InjectedStore
- ToolNode

**Functions:**

- msg_content_output
- tools_condition

**Constants / Objects:**

- AnyMessage
- Callable
- INVALID_TOOL_NAME_ERROR_TEMPLATE
- Literal
- Optional
- REMOVE_ALL_MESSAGES
- Sequence
- TOOL_CALL_ERROR_TEMPLATE
- TOOL_MESSAGE_BLOCK_TYPES
- Tuple
- Type
- Union

**Import statements:**

- `from langgraph.prebuilt.tool_node import InjectedState`
- `from langgraph.prebuilt.tool_node import InjectedStore`
- `from langgraph.prebuilt.tool_node import ToolNode`
- `from langgraph.prebuilt.tool_node import msg_content_output`
- `from langgraph.prebuilt.tool_node import tools_condition`


## Module: `langgraph.prebuilt.tool_validator`

**Classes:**

- ValidationNode

**Constants / Objects:**

- AnyMessage
- Callable
- Dict
- Optional
- Sequence
- Tuple
- Type
- Union

**Import statements:**

- `from langgraph.prebuilt.tool_validator import ValidationNode`

In [8]:
import importlib, inspect, pkgutil
from types import ModuleType
from typing import Optional
from IPython.display import Markdown, display

def _is_public_segment(seg: str) -> bool:
    # exclude names like "_retry" or "__something__"
    return not seg.startswith("_")

def _is_public_module_name(modname: str) -> bool:
    # every segment must be public
    return all(_is_public_segment(seg) for seg in modname.split("."))

def _load_module(name: str) -> Optional[ModuleType]:
    try:
        return importlib.import_module(name)
    except Exception:
        return None

def _iter_public_submodules(pkg_name: str):
    """Yield (name, module) for pkg and all submodules, skipping any module
    that has a private segment (e.g., langgraph.pregel._retry)."""
    pkg = _load_module(pkg_name)
    if not pkg or not hasattr(pkg, "__path__"):
        return
    if _is_public_module_name(pkg_name):
        yield pkg_name, pkg
    for m in pkgutil.walk_packages(pkg.__path__, prefix=pkg_name + "."):
        if not _is_public_module_name(m.name):
            continue
        mod = _load_module(m.name)
        if mod is not None:
            yield m.name, mod

def _public_names(mod: ModuleType):
    # prefer __all__ (captures re-exports), else dir()
    names = getattr(mod, "__all__", None)
    if names is None:
        names = [n for n in dir(mod) if not n.startswith("_")]
    else:
        names = [n for n in names if not n.startswith("_")]
    return names

def _kind_of(obj):
    if inspect.isclass(obj): return "class"
    if inspect.isfunction(obj) or inspect.ismethod(obj) or inspect.isbuiltin(obj): return "function"
    if inspect.ismodule(obj): return "module"
    return "const"

def dump_api_markdown(root: str = "langgraph",
                      include_const_imports: bool = True,
                      show_origin_for_reexports: bool = True) -> str:
    lines = []
    for modname, mod in _iter_public_submodules(root):
        names = _public_names(mod)
        classes, funcs, consts = [], [], []
        for n in sorted(names):
            try:
                obj = getattr(mod, n)
            except Exception:
                continue
            k = _kind_of(obj)
            if k == "class": classes.append((n, obj))
            elif k == "function": funcs.append((n, obj))
            elif k == "const": consts.append((n, obj))
        if not (classes or funcs or consts):
            continue

        lines.append(f"### Module: `{modname}`\n")

        if classes:
            lines.append("**Classes:**")
            for name, obj in classes:
                origin = getattr(obj, "__module__", None)
                tag = f"  *(from `{origin}`)*" if (show_origin_for_reexports and origin and origin != modname) else ""
                lines.append(f"- {name}{tag}")
            lines.append("")

        if funcs:
            lines.append("**Functions:**")
            for name, obj in funcs:
                origin = getattr(obj, "__module__", None)
                tag = f"  *(from `{origin}`)*" if (show_origin_for_reexports and origin and origin != modname) else ""
                lines.append(f"- {name}{tag}")
            lines.append("")

        if consts:
            lines.append("**Constants / Objects / Sentinels:**")
            for name, obj in consts:
                lines.append(f"- {name}  *(type: `{type(obj).__name__}`)*")
            lines.append("")

        imports = [f"from {modname} import {name}" for name, _ in classes + funcs]
        if include_const_imports:
            imports += [f"from {modname} import {name}" for name, _ in consts]
        if imports:
            lines.append("**Import statements:**")
            lines += [f"- `{imp}`" for imp in imports]
            lines.append("")
        lines.append("")
    return "\n".join(lines).strip()


# EXAMPLES:
# 1) Whole package
md = dump_api_markdown("langgraph", include_const_imports=True, show_origin_for_reexports=True)
display(Markdown(md))

# 2) Focus on a subpackage:
# md2 = dump_api_markdown("langgraph.graph", include_const_imports=True, show_origin_for_reexports=True)
# display(Markdown(md2))


### Module: `langgraph.channels`

**Classes:**
- AnyValue  *(from `langgraph.channels.any_value`)*
- BaseChannel  *(from `langgraph.channels.base`)*
- BinaryOperatorAggregate  *(from `langgraph.channels.binop`)*
- EphemeralValue  *(from `langgraph.channels.ephemeral_value`)*
- LastValue  *(from `langgraph.channels.last_value`)*
- LastValueAfterFinish  *(from `langgraph.channels.last_value`)*
- NamedBarrierValue  *(from `langgraph.channels.named_barrier_value`)*
- NamedBarrierValueAfterFinish  *(from `langgraph.channels.named_barrier_value`)*
- Topic  *(from `langgraph.channels.topic`)*
- UntrackedValue  *(from `langgraph.channels.untracked_value`)*

**Import statements:**
- `from langgraph.channels import AnyValue`
- `from langgraph.channels import BaseChannel`
- `from langgraph.channels import BinaryOperatorAggregate`
- `from langgraph.channels import EphemeralValue`
- `from langgraph.channels import LastValue`
- `from langgraph.channels import LastValueAfterFinish`
- `from langgraph.channels import NamedBarrierValue`
- `from langgraph.channels import NamedBarrierValueAfterFinish`
- `from langgraph.channels import Topic`
- `from langgraph.channels import UntrackedValue`


### Module: `langgraph.channels.any_value`

**Classes:**
- AnyValue

**Import statements:**
- `from langgraph.channels.any_value import AnyValue`


### Module: `langgraph.channels.base`

**Classes:**
- BaseChannel

**Import statements:**
- `from langgraph.channels.base import BaseChannel`


### Module: `langgraph.channels.binop`

**Classes:**
- BinaryOperatorAggregate

**Import statements:**
- `from langgraph.channels.binop import BinaryOperatorAggregate`


### Module: `langgraph.channels.ephemeral_value`

**Classes:**
- EphemeralValue

**Import statements:**
- `from langgraph.channels.ephemeral_value import EphemeralValue`


### Module: `langgraph.channels.last_value`

**Classes:**
- LastValue
- LastValueAfterFinish

**Import statements:**
- `from langgraph.channels.last_value import LastValue`
- `from langgraph.channels.last_value import LastValueAfterFinish`


### Module: `langgraph.channels.named_barrier_value`

**Classes:**
- NamedBarrierValue
- NamedBarrierValueAfterFinish

**Import statements:**
- `from langgraph.channels.named_barrier_value import NamedBarrierValue`
- `from langgraph.channels.named_barrier_value import NamedBarrierValueAfterFinish`


### Module: `langgraph.channels.topic`

**Classes:**
- Topic

**Import statements:**
- `from langgraph.channels.topic import Topic`


### Module: `langgraph.channels.untracked_value`

**Classes:**
- UntrackedValue

**Import statements:**
- `from langgraph.channels.untracked_value import UntrackedValue`


### Module: `langgraph.config`

**Classes:**
- Any  *(from `typing`)*
- BaseStore  *(from `langgraph.store.base`)*
- RunnableConfig  *(from `langchain_core.runnables.config`)*

**Functions:**
- get_config
- get_store
- get_stream_writer

**Constants / Objects / Sentinels:**
- CONF  *(type: `str`)*
- CONFIG_KEY_RUNTIME  *(type: `str`)*
- StreamWriter  *(type: `_CallableGenericAlias`)*
- var_child_runnable_config  *(type: `ContextVar`)*

**Import statements:**
- `from langgraph.config import Any`
- `from langgraph.config import BaseStore`
- `from langgraph.config import RunnableConfig`
- `from langgraph.config import get_config`
- `from langgraph.config import get_store`
- `from langgraph.config import get_stream_writer`
- `from langgraph.config import CONF`
- `from langgraph.config import CONFIG_KEY_RUNTIME`
- `from langgraph.config import StreamWriter`
- `from langgraph.config import var_child_runnable_config`


### Module: `langgraph.constants`

**Constants / Objects / Sentinels:**
- CONF  *(type: `str`)*
- CONFIG_KEY_CHECKPOINTER  *(type: `str`)*
- END  *(type: `str`)*
- START  *(type: `str`)*
- TAG_HIDDEN  *(type: `str`)*
- TAG_NOSTREAM  *(type: `str`)*
- TASKS  *(type: `str`)*

**Import statements:**
- `from langgraph.constants import CONF`
- `from langgraph.constants import CONFIG_KEY_CHECKPOINTER`
- `from langgraph.constants import END`
- `from langgraph.constants import START`
- `from langgraph.constants import TAG_HIDDEN`
- `from langgraph.constants import TAG_NOSTREAM`
- `from langgraph.constants import TASKS`


### Module: `langgraph.errors`

**Classes:**
- EmptyChannelError  *(from `langgraph.checkpoint.base`)*
- EmptyInputError
- ErrorCode
- GraphBubbleUp
- GraphInterrupt
- GraphRecursionError
- InvalidUpdateError
- NodeInterrupt
- ParentCommand
- TaskNotFound

**Import statements:**
- `from langgraph.errors import EmptyChannelError`
- `from langgraph.errors import EmptyInputError`
- `from langgraph.errors import ErrorCode`
- `from langgraph.errors import GraphBubbleUp`
- `from langgraph.errors import GraphInterrupt`
- `from langgraph.errors import GraphRecursionError`
- `from langgraph.errors import InvalidUpdateError`
- `from langgraph.errors import NodeInterrupt`
- `from langgraph.errors import ParentCommand`
- `from langgraph.errors import TaskNotFound`


### Module: `langgraph.func`

**Classes:**
- entrypoint

**Functions:**
- task

**Import statements:**
- `from langgraph.func import entrypoint`
- `from langgraph.func import task`


### Module: `langgraph.graph`

**Classes:**
- MessageGraph  *(from `langgraph.graph.message`)*
- MessagesState  *(from `langgraph.graph.message`)*
- StateGraph  *(from `langgraph.graph.state`)*

**Functions:**
- add_messages  *(from `langgraph.graph.message`)*

**Constants / Objects / Sentinels:**
- END  *(type: `str`)*
- START  *(type: `str`)*

**Import statements:**
- `from langgraph.graph import MessageGraph`
- `from langgraph.graph import MessagesState`
- `from langgraph.graph import StateGraph`
- `from langgraph.graph import add_messages`
- `from langgraph.graph import END`
- `from langgraph.graph import START`


### Module: `langgraph.graph.message`

**Classes:**
- MessageGraph
- MessagesState

**Functions:**
- add_messages

**Import statements:**
- `from langgraph.graph.message import MessageGraph`
- `from langgraph.graph.message import MessagesState`
- `from langgraph.graph.message import add_messages`


### Module: `langgraph.graph.state`

**Classes:**
- CompiledStateGraph
- StateGraph

**Import statements:**
- `from langgraph.graph.state import CompiledStateGraph`
- `from langgraph.graph.state import StateGraph`


### Module: `langgraph.graph.ui`

**Classes:**
- RemoveUIMessage
- UIMessage

**Functions:**
- delete_ui_message
- push_ui_message
- ui_message_reducer

**Constants / Objects / Sentinels:**
- AnyUIMessage  *(type: `_UnionGenericAlias`)*

**Import statements:**
- `from langgraph.graph.ui import RemoveUIMessage`
- `from langgraph.graph.ui import UIMessage`
- `from langgraph.graph.ui import delete_ui_message`
- `from langgraph.graph.ui import push_ui_message`
- `from langgraph.graph.ui import ui_message_reducer`
- `from langgraph.graph.ui import AnyUIMessage`


### Module: `langgraph.managed`

**Constants / Objects / Sentinels:**
- IsLastStep  *(type: `_AnnotatedAlias`)*
- RemainingSteps  *(type: `_AnnotatedAlias`)*

**Import statements:**
- `from langgraph.managed import IsLastStep`
- `from langgraph.managed import RemainingSteps`


### Module: `langgraph.managed.base`

**Constants / Objects / Sentinels:**
- ManagedValueMapping  *(type: `GenericAlias`)*
- ManagedValueSpec  *(type: `GenericAlias`)*

**Import statements:**
- `from langgraph.managed.base import ManagedValueMapping`
- `from langgraph.managed.base import ManagedValueSpec`


### Module: `langgraph.managed.is_last_step`

**Classes:**
- RemainingStepsManager

**Constants / Objects / Sentinels:**
- IsLastStep  *(type: `_AnnotatedAlias`)*

**Import statements:**
- `from langgraph.managed.is_last_step import RemainingStepsManager`
- `from langgraph.managed.is_last_step import IsLastStep`


### Module: `langgraph.prebuilt`

**Classes:**
- InjectedState  *(from `langgraph.prebuilt.tool_node`)*
- InjectedStore  *(from `langgraph.prebuilt.tool_node`)*
- ToolNode  *(from `langgraph.prebuilt.tool_node`)*
- ValidationNode  *(from `langgraph.prebuilt.tool_validator`)*

**Functions:**
- create_react_agent  *(from `langgraph.prebuilt.chat_agent_executor`)*
- tools_condition  *(from `langgraph.prebuilt.tool_node`)*

**Import statements:**
- `from langgraph.prebuilt import InjectedState`
- `from langgraph.prebuilt import InjectedStore`
- `from langgraph.prebuilt import ToolNode`
- `from langgraph.prebuilt import ValidationNode`
- `from langgraph.prebuilt import create_react_agent`
- `from langgraph.prebuilt import tools_condition`


### Module: `langgraph.prebuilt.chat_agent_executor`

**Classes:**
- AgentState
- AgentStatePydantic
- AgentStateWithStructuredResponse
- AgentStateWithStructuredResponsePydantic

**Functions:**
- create_react_agent
- create_tool_calling_executor

**Import statements:**
- `from langgraph.prebuilt.chat_agent_executor import AgentState`
- `from langgraph.prebuilt.chat_agent_executor import AgentStatePydantic`
- `from langgraph.prebuilt.chat_agent_executor import AgentStateWithStructuredResponse`
- `from langgraph.prebuilt.chat_agent_executor import AgentStateWithStructuredResponsePydantic`
- `from langgraph.prebuilt.chat_agent_executor import create_react_agent`
- `from langgraph.prebuilt.chat_agent_executor import create_tool_calling_executor`


### Module: `langgraph.prebuilt.interrupt`

**Classes:**
- ActionRequest
- HumanInterrupt
- HumanInterruptConfig
- HumanResponse

**Constants / Objects / Sentinels:**
- Literal  *(type: `_LiteralSpecialForm`)*
- Optional  *(type: `_SpecialForm`)*
- TypedDict  *(type: `_TypedDictSpecialForm`)*
- Union  *(type: `_SpecialForm`)*

**Import statements:**
- `from langgraph.prebuilt.interrupt import ActionRequest`
- `from langgraph.prebuilt.interrupt import HumanInterrupt`
- `from langgraph.prebuilt.interrupt import HumanInterruptConfig`
- `from langgraph.prebuilt.interrupt import HumanResponse`
- `from langgraph.prebuilt.interrupt import Literal`
- `from langgraph.prebuilt.interrupt import Optional`
- `from langgraph.prebuilt.interrupt import TypedDict`
- `from langgraph.prebuilt.interrupt import Union`


### Module: `langgraph.prebuilt.tool_node`

**Classes:**
- AIMessage  *(from `langchain_core.messages.ai`)*
- Annotated  *(from `typing`)*
- Any  *(from `typing`)*
- BaseModel  *(from `pydantic.main`)*
- BaseStore  *(from `langgraph.store.base`)*
- BaseTool  *(from `langchain_core.tools.base`)*
- Command  *(from `langgraph.types`)*
- GraphBubbleUp  *(from `langgraph.errors`)*
- InjectedState
- InjectedStore
- InjectedToolArg  *(from `langchain_core.tools.base`)*
- RemoveMessage  *(from `langchain_core.messages.modifier`)*
- RunnableCallable  *(from `langgraph._internal._runnable`)*
- RunnableConfig  *(from `langchain_core.runnables.config`)*
- Send  *(from `langgraph.types`)*
- ToolCall  *(from `langchain_core.messages.tool`)*
- ToolMessage  *(from `langchain_core.messages.tool`)*
- ToolNode

**Functions:**
- cast  *(from `typing`)*
- convert_to_messages  *(from `langchain_core.messages.utils`)*
- copy  *(from `copy`)*
- create_tool  *(from `langchain_core.tools.convert`)*
- deepcopy  *(from `copy`)*
- get_all_basemodel_annotations  *(from `langchain_core.tools.base`)*
- get_args  *(from `typing`)*
- get_config_list  *(from `langchain_core.runnables.config`)*
- get_executor_for_config  *(from `langchain_core.runnables.config`)*
- get_origin  *(from `typing`)*
- get_type_hints  *(from `typing`)*
- msg_content_output
- replace  *(from `dataclasses`)*
- tools_condition

**Constants / Objects / Sentinels:**
- AnyMessage  *(type: `_AnnotatedAlias`)*
- Callable  *(type: `_CallableType`)*
- INVALID_TOOL_NAME_ERROR_TEMPLATE  *(type: `str`)*
- Literal  *(type: `_LiteralSpecialForm`)*
- Optional  *(type: `_SpecialForm`)*
- REMOVE_ALL_MESSAGES  *(type: `str`)*
- Sequence  *(type: `_SpecialGenericAlias`)*
- TOOL_CALL_ERROR_TEMPLATE  *(type: `str`)*
- TOOL_MESSAGE_BLOCK_TYPES  *(type: `tuple`)*
- Tuple  *(type: `_TupleType`)*
- Type  *(type: `_SpecialGenericAlias`)*
- Union  *(type: `_SpecialForm`)*

**Import statements:**
- `from langgraph.prebuilt.tool_node import AIMessage`
- `from langgraph.prebuilt.tool_node import Annotated`
- `from langgraph.prebuilt.tool_node import Any`
- `from langgraph.prebuilt.tool_node import BaseModel`
- `from langgraph.prebuilt.tool_node import BaseStore`
- `from langgraph.prebuilt.tool_node import BaseTool`
- `from langgraph.prebuilt.tool_node import Command`
- `from langgraph.prebuilt.tool_node import GraphBubbleUp`
- `from langgraph.prebuilt.tool_node import InjectedState`
- `from langgraph.prebuilt.tool_node import InjectedStore`
- `from langgraph.prebuilt.tool_node import InjectedToolArg`
- `from langgraph.prebuilt.tool_node import RemoveMessage`
- `from langgraph.prebuilt.tool_node import RunnableCallable`
- `from langgraph.prebuilt.tool_node import RunnableConfig`
- `from langgraph.prebuilt.tool_node import Send`
- `from langgraph.prebuilt.tool_node import ToolCall`
- `from langgraph.prebuilt.tool_node import ToolMessage`
- `from langgraph.prebuilt.tool_node import ToolNode`
- `from langgraph.prebuilt.tool_node import cast`
- `from langgraph.prebuilt.tool_node import convert_to_messages`
- `from langgraph.prebuilt.tool_node import copy`
- `from langgraph.prebuilt.tool_node import create_tool`
- `from langgraph.prebuilt.tool_node import deepcopy`
- `from langgraph.prebuilt.tool_node import get_all_basemodel_annotations`
- `from langgraph.prebuilt.tool_node import get_args`
- `from langgraph.prebuilt.tool_node import get_config_list`
- `from langgraph.prebuilt.tool_node import get_executor_for_config`
- `from langgraph.prebuilt.tool_node import get_origin`
- `from langgraph.prebuilt.tool_node import get_type_hints`
- `from langgraph.prebuilt.tool_node import msg_content_output`
- `from langgraph.prebuilt.tool_node import replace`
- `from langgraph.prebuilt.tool_node import tools_condition`
- `from langgraph.prebuilt.tool_node import AnyMessage`
- `from langgraph.prebuilt.tool_node import Callable`
- `from langgraph.prebuilt.tool_node import INVALID_TOOL_NAME_ERROR_TEMPLATE`
- `from langgraph.prebuilt.tool_node import Literal`
- `from langgraph.prebuilt.tool_node import Optional`
- `from langgraph.prebuilt.tool_node import REMOVE_ALL_MESSAGES`
- `from langgraph.prebuilt.tool_node import Sequence`
- `from langgraph.prebuilt.tool_node import TOOL_CALL_ERROR_TEMPLATE`
- `from langgraph.prebuilt.tool_node import TOOL_MESSAGE_BLOCK_TYPES`
- `from langgraph.prebuilt.tool_node import Tuple`
- `from langgraph.prebuilt.tool_node import Type`
- `from langgraph.prebuilt.tool_node import Union`


### Module: `langgraph.prebuilt.tool_validator`

**Classes:**
- AIMessage  *(from `langchain_core.messages.ai`)*
- Any  *(from `typing`)*
- BaseModel  *(from `pydantic.main`)*
- BaseModelV1  *(from `pydantic.v1.main`)*
- BaseTool  *(from `langchain_core.tools.base`)*
- RunnableCallable  *(from `langgraph._internal._runnable`)*
- RunnableConfig  *(from `langchain_core.runnables.config`)*
- ToolCall  *(from `langchain_core.messages.tool`)*
- ToolMessage  *(from `langchain_core.messages.tool`)*
- ValidationError  *(from `pydantic_core._pydantic_core`)*
- ValidationErrorV1  *(from `pydantic.v1.error_wrappers`)*
- ValidationNode

**Functions:**
- cast  *(from `typing`)*
- create_schema_from_function  *(from `langchain_core.tools.base`)*
- get_executor_for_config  *(from `langchain_core.runnables.config`)*
- is_basemodel_subclass  *(from `langchain_core.utils.pydantic`)*

**Constants / Objects / Sentinels:**
- AnyMessage  *(type: `_AnnotatedAlias`)*
- Callable  *(type: `_CallableType`)*
- Dict  *(type: `_SpecialGenericAlias`)*
- Optional  *(type: `_SpecialForm`)*
- Sequence  *(type: `_SpecialGenericAlias`)*
- Tuple  *(type: `_TupleType`)*
- Type  *(type: `_SpecialGenericAlias`)*
- Union  *(type: `_SpecialForm`)*

**Import statements:**
- `from langgraph.prebuilt.tool_validator import AIMessage`
- `from langgraph.prebuilt.tool_validator import Any`
- `from langgraph.prebuilt.tool_validator import BaseModel`
- `from langgraph.prebuilt.tool_validator import BaseModelV1`
- `from langgraph.prebuilt.tool_validator import BaseTool`
- `from langgraph.prebuilt.tool_validator import RunnableCallable`
- `from langgraph.prebuilt.tool_validator import RunnableConfig`
- `from langgraph.prebuilt.tool_validator import ToolCall`
- `from langgraph.prebuilt.tool_validator import ToolMessage`
- `from langgraph.prebuilt.tool_validator import ValidationError`
- `from langgraph.prebuilt.tool_validator import ValidationErrorV1`
- `from langgraph.prebuilt.tool_validator import ValidationNode`
- `from langgraph.prebuilt.tool_validator import cast`
- `from langgraph.prebuilt.tool_validator import create_schema_from_function`
- `from langgraph.prebuilt.tool_validator import get_executor_for_config`
- `from langgraph.prebuilt.tool_validator import is_basemodel_subclass`
- `from langgraph.prebuilt.tool_validator import AnyMessage`
- `from langgraph.prebuilt.tool_validator import Callable`
- `from langgraph.prebuilt.tool_validator import Dict`
- `from langgraph.prebuilt.tool_validator import Optional`
- `from langgraph.prebuilt.tool_validator import Sequence`
- `from langgraph.prebuilt.tool_validator import Tuple`
- `from langgraph.prebuilt.tool_validator import Type`
- `from langgraph.prebuilt.tool_validator import Union`


### Module: `langgraph.pregel`

**Classes:**
- NodeBuilder  *(from `langgraph.pregel.main`)*
- Pregel  *(from `langgraph.pregel.main`)*

**Import statements:**
- `from langgraph.pregel import NodeBuilder`
- `from langgraph.pregel import Pregel`


### Module: `langgraph.pregel.debug`

**Classes:**
- CheckpointPayload
- CheckpointTask
- TaskPayload
- TaskResultPayload

**Import statements:**
- `from langgraph.pregel.debug import CheckpointPayload`
- `from langgraph.pregel.debug import CheckpointTask`
- `from langgraph.pregel.debug import TaskPayload`
- `from langgraph.pregel.debug import TaskResultPayload`


### Module: `langgraph.pregel.main`

**Classes:**
- NodeBuilder
- Pregel

**Import statements:**
- `from langgraph.pregel.main import NodeBuilder`
- `from langgraph.pregel.main import Pregel`


### Module: `langgraph.pregel.protocol`

**Classes:**
- PregelProtocol
- StreamProtocol

**Import statements:**
- `from langgraph.pregel.protocol import PregelProtocol`
- `from langgraph.pregel.protocol import StreamProtocol`


### Module: `langgraph.pregel.remote`

**Classes:**
- RemoteException
- RemoteGraph

**Import statements:**
- `from langgraph.pregel.remote import RemoteException`
- `from langgraph.pregel.remote import RemoteGraph`


### Module: `langgraph.pregel.types`

**Classes:**
- CachePolicy  *(from `langgraph.types`)*
- PregelExecutableTask  *(from `langgraph.types`)*
- PregelTask  *(from `langgraph.types`)*
- RetryPolicy  *(from `langgraph.types`)*
- StateSnapshot  *(from `langgraph.types`)*
- StateUpdate  *(from `langgraph.types`)*

**Functions:**
- default_retry_on  *(from `langgraph._internal._retry`)*

**Constants / Objects / Sentinels:**
- All  *(type: `_LiteralGenericAlias`)*
- StreamMode  *(type: `_LiteralGenericAlias`)*
- StreamWriter  *(type: `_CallableGenericAlias`)*

**Import statements:**
- `from langgraph.pregel.types import CachePolicy`
- `from langgraph.pregel.types import PregelExecutableTask`
- `from langgraph.pregel.types import PregelTask`
- `from langgraph.pregel.types import RetryPolicy`
- `from langgraph.pregel.types import StateSnapshot`
- `from langgraph.pregel.types import StateUpdate`
- `from langgraph.pregel.types import default_retry_on`
- `from langgraph.pregel.types import All`
- `from langgraph.pregel.types import StreamMode`
- `from langgraph.pregel.types import StreamWriter`


### Module: `langgraph.runtime`

**Classes:**
- Runtime

**Functions:**
- get_runtime

**Import statements:**
- `from langgraph.runtime import Runtime`
- `from langgraph.runtime import get_runtime`


### Module: `langgraph.types`

**Classes:**
- CachePolicy
- Command
- Interrupt
- PregelExecutableTask
- PregelTask
- RetryPolicy
- Send
- StateSnapshot
- StateUpdate

**Functions:**
- interrupt

**Constants / Objects / Sentinels:**
- All  *(type: `_LiteralGenericAlias`)*
- Checkpointer  *(type: `_UnionGenericAlias`)*
- Durability  *(type: `_LiteralGenericAlias`)*
- StreamMode  *(type: `_LiteralGenericAlias`)*
- StreamWriter  *(type: `_CallableGenericAlias`)*

**Import statements:**
- `from langgraph.types import CachePolicy`
- `from langgraph.types import Command`
- `from langgraph.types import Interrupt`
- `from langgraph.types import PregelExecutableTask`
- `from langgraph.types import PregelTask`
- `from langgraph.types import RetryPolicy`
- `from langgraph.types import Send`
- `from langgraph.types import StateSnapshot`
- `from langgraph.types import StateUpdate`
- `from langgraph.types import interrupt`
- `from langgraph.types import All`
- `from langgraph.types import Checkpointer`
- `from langgraph.types import Durability`
- `from langgraph.types import StreamMode`
- `from langgraph.types import StreamWriter`


### Module: `langgraph.typing`

**Constants / Objects / Sentinels:**
- ContextT  *(type: `TypeVar`)*
- InputT  *(type: `TypeVar`)*
- OutputT  *(type: `TypeVar`)*
- StateT  *(type: `TypeVar`)*
- StateT_co  *(type: `TypeVar`)*
- StateT_contra  *(type: `TypeVar`)*

**Import statements:**
- `from langgraph.typing import ContextT`
- `from langgraph.typing import InputT`
- `from langgraph.typing import OutputT`
- `from langgraph.typing import StateT`
- `from langgraph.typing import StateT_co`
- `from langgraph.typing import StateT_contra`


### Module: `langgraph.utils.config`

**Functions:**
- ensure_config  *(from `langgraph._internal._config`)*
- get_config  *(from `langgraph.config`)*
- get_store  *(from `langgraph.config`)*
- patch_configurable  *(from `langgraph._internal._config`)*

**Import statements:**
- `from langgraph.utils.config import ensure_config`
- `from langgraph.utils.config import get_config`
- `from langgraph.utils.config import get_store`
- `from langgraph.utils.config import patch_configurable`


### Module: `langgraph.utils.runnable`

**Classes:**
- RunnableCallable  *(from `langgraph._internal._runnable`)*

**Constants / Objects / Sentinels:**
- RunnableLike  *(type: `_UnionGenericAlias`)*

**Import statements:**
- `from langgraph.utils.runnable import RunnableCallable`
- `from langgraph.utils.runnable import RunnableLike`


### Module: `langgraph.warnings`

**Classes:**
- LangGraphDeprecatedSinceV05
- LangGraphDeprecatedSinceV10
- LangGraphDeprecationWarning

**Import statements:**
- `from langgraph.warnings import LangGraphDeprecatedSinceV05`
- `from langgraph.warnings import LangGraphDeprecatedSinceV10`
- `from langgraph.warnings import LangGraphDeprecationWarning`